# AuxTel - Sky Flats Sequence

This notebook is a draft of procedures we can use during twilight to take Sky Flats using different filters as per SITCOM-790. 
https://jira.lsstcorp.org/browse/SITCOM-790https://jira.lsstcorp.org/browse/SITCOM-790

It is meant to be executed between sunset and the time between the end of the civil and nautical twilights, that is, when the Sun's elevation is between 0 and 9 degrees below horizon. At 9 degrees below horizon, AuxTel should go on-sky to absorb zero-point offsets and perform the first WEP mirror alignment and focus of the night.

This procedure requires that AuxTel is ready for on-sky in advance, when the Sun is 25 degrees above horizon. 

Be prepared to run this notebook at the time of sunset to have enough time for the three available filters. 

After the setup is completed, we find an empty field in the opposite direction of the Sun, then we slew to this field, and take an image. This image will probably have saturation. We start a loop of taking images in the same position until we get an image that does not saturate. Also start with the bluest filter. 

Once the image is not saturated anymore, we take a sequence of images with small dithers and increasing exposure times. SITCOM-something like 12 flats per filter.

## Setup

In [51]:
import asyncio
import time
import os
import numpy as np
import matplotlib.pyplot as plt
import logging
import time

from astropy import coordinates
from astropy import units as u
from astropy.time import Time
from astroquery.exceptions import NoResultsWarning
from astroquery.vizier import Vizier
from datetime import datetime
from warnings import warn

from lsst.ts import salobj

from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS
from lsst.ts.observatory.control.utils import RotType

import lsst.daf.butler.cli.cliLog as cliLog
import lsst.daf.butler as dafButler

In [52]:
logger = logging.getLogger(f"Sky Flats {Time.now()} UT")
logger.level = logging.INFO

In [53]:
# Instantiate the control classes
domain = salobj.Domain()
atcs = ATCS(domain)
atcs.set_rem_loglevel(logging.INFO)
latiss = LATISS(domain)
latiss.set_rem_loglevel(logging.INFO)
await asyncio.gather(atcs.start_task, latiss.start_task)

[[None, None, None, None, None, None, None], [None, None, None, None]]

In [54]:
inst_setup = await latiss.get_available_instrument_setup()
logger.info(f'LATISS filters are: {inst_setup[0]}')
logger.info(f'LATISS gratings are: {inst_setup[1]}')

In [55]:
repo = '/repo/LATISS/'
collection = 'LATISS/raw/all'

butler = dafButler.Butler(repo, collections=collection)

In [49]:
# Vera Rubin Coordinates from https://www.lsst.org/scientists/keynumbers
rubin_obs = coordinates.EarthLocation(
    lat='-30:14:40.68', lon='-70:44:57.90', height=2647 * u.m)

## Helper functions 

In [45]:
def get_angle_from_sun(distance_from_sun=180, elevation=45, time=None):
    """
    Returns the azimuth relative to the Sun considering a `distance_from_sun` 
    and at a given `time`.
    
    Parameters
    ----------
    distance_from_sun : float, (-180, 180)
        The distance from the Sun in degrees. Positive angles go towards the 
        North.
    elevation : float
        Target elevation for Sky Flats.
    time : datetime
        The time for the calculation in UTC.
    """
    if abs(distance_from_sun) > 180:
        raise ValueError(
            "The distance from the Sun should be between -180 and 180 degrees")
    
    rubin_obs = coordinates.EarthLocation(
        lat='-30:14:40.68', lon='-70:44:57.90', height=2647 * u.m)

    if time:
        t = Time(time, scale='utc')
    else:
        t = Time(datetime.utcnow(), scale='utc')

    sun_coords = coordinates.get_sun(t)
    sun_coords.location = rubin_obs
    az = sun_coords.altaz.az.value
    
    target_az = (az + distance_from_sun) % 360.
    
    logger = DecoratedLogger.get_decorated_logger('sun_position')
    logger.info(f"Sun azimuth is {sun_coords.altaz.az:.5f} at {t}")
    logger.info(f"The azimuth {distance_from_sun:.5f} deg"
                f" from the Sun is {target_az:.5f} deg")
    
    target = coordinates.AltAz(
        alt=elevation * u.deg, az=target_az * u.deg, location=sun_coords, obstime=t)
    
    return target.transform_to(coordinates.ICRS)


In [46]:
def get_empty_field(target, radius=5):
    """
    Query the "Deep blank field catalogue : J/MNRAS/427/679" in Vizier.
    
    Parameters
    ----------
    target : astropy.coordinates.SkyCoord
        Sky coordinates near the field
    radius : float
        Search radius in degrees.
    elevation : float, default: 45
        Start elevation for the Sky Flats.
    
    Reference
    ---------
    http://cdsarc.u-strasbg.fr/viz-bin/Cat?J/MNRAS/427/679
    """
    _table = Vizier.query_region(
        catalog='J/MNRAS/427/679/blank_fld', 
        coordinates=target, 
        radius=radius*u.deg)

    if len(_table) == 0:
        warn(f"Could not find a field near {target} "
              f"within {radius} deg radius", category=NoResultsWarning)
        return None

    _table = _table["J/MNRAS/427/679/blank_fld"]
    
    coords = coordinates.SkyCoord(
        ra=_table["RAJ2000"], 
        dec=_table["DEJ2000"], 
        unit=(u.hourangle, u.deg), 
        frame=coordinates.ICRS)
    
    arg = target.separation(coords).argmin()
    
    return coords[arg]

## Confirm Sun's elevation is right to start taking Sky Flats. 

Check if we can start taking sky flats, that, if the Sun's elevation is between 0 and 9 degrees below the horizon.

In [57]:
# Define time in UTC
#time = Time("2023-04-25T17:00:00", format="isot", scale="utc")
# or the current time
time = Time(datetime.utcnow(), scale="utc")

# Some calculations
sun_coordinates = coordinates.get_sun(time)
sun_coordinates.location = rubin_obs
where_sun = "setting" if (sun_coordinates.altaz.az.value > 180) else "rising"

# Print results
print(f" The azimuth of the {where_sun} Sun at {time} is {sun_coordinates.altaz.az.value:.2f} deg \n"
      f" The elevation of the Sun at {time} is {sun_coordinates.altaz.alt.value:.2f} deg")

 The azimuth of the setting Sun at 2023-05-02 20:39:32.540372 is 299.64 deg 
 The elevation of the Sun at 2023-05-02 20:39:32.540372 is 16.44 deg


Only proceed with the next steps if the elevation of the Sun is between 0 and 9 degrees below the horizon, either rising or setting. 

## Find Field

Find a field with not many stars oppposite the Sun. 

In [42]:
target = get_angle_from_sun()
empty_field_coords = get_empty_field(target)
print(f"Empty field coordinates:\n"
      f"  {empty_field_coords.ra.to_string(u.hour, sep=':')}"
      f" {empty_field_coords.dec.to_string(u.degree, alwayssign=True, sep=':')}")

  [I 2023-05-02 18:03:34 sun_position]  Sun azimuth is 333.30924 deg at 2023-05-02 18:03:34.759041


  [I 2023-05-02 18:03:34 sun_position]  The azimuth 180.00000 deg from the Sun is 153.30924 deg


Empty field coordinates:
  3:32:42.02 -24:44:56.8


## Slew to Field

Now that we have our field, let's move the telescope and the dome to that direction.

In [ ]:
# await atcs.slew_icrs(empty_field_coords.ra.value, empty_field_coords.dec.value)

## Define filter and offsets between images

Need at least 10K ADU per pixel for each band.

In [24]:
filter_to_use = 'SDSSg_65mm'
offset = 30

## Find first image

Here is where we check image saturation. We take an image with some short exposure time, check how many pixels on each detector are saturated, check how many counts do we have in each detector, and once we have high but not saturated signal is all detectors and in most of the pixels, we move forward.

In [33]:
latiss.rem.atoods.evt_imageInOODS.flush()
# Take image
test_exp = await latiss.take_flats(
    exptime=1, nflats=1, filter=filter_to_use, grating='empty_1', reason="Test", program="SITCOM-790")
print(f"The test exposure is {test_exp[0]}")

# Wait for image ingestion
await latiss.rem.atoods.evt_imageInOODS.next(flush=False, timeout=10)

# Measure median, mean and std on the image
dataId = {'detector': 0, 'exposure': test_exp[0]}
exp = butler.get('raw', dataId=dataId)     
foo = exp.getMaskedImage()
masked_array = np.ma.masked_array(foo.image.array, mask=foo.mask.array)

logger.info(f"Observation ID: {level[0]}")
logger.info(f"    Median: {np.ma.median(masked_array)}")
logger.info(f"    Mean: {np.ma.mean(masked_array)}")
logger.info(f"    Std: {np.ma.std(masked_array)}")

[2023050200007]


## Start Sequence

Now that we have our start point for the filter, we take triplets of sky flats with offsets of 30 arcsec between each. We will repeat with the different exposure times defined in the `exp_times` array in the next cell. 

Confirm signal levels as they show up in the http://ccs.lsst.org/RecentImages/auxtel.htmlhttp://ccs.lsst.org/RecentImages/auxtel.html.

In [ ]:
exp_times = [2,4,8,16]

In [37]:
for time in exp_times:
    print(f"Starting with {time} seconds sky-flats exposures")
    
    await atcs.offset_xy(x=0, y=0)
    await latiss.take_flats(
        exptime=time, nflats=1, filter=filter_to_use, grating='empty_1', reason="Sky_Flat", program="Test-790")
    
    await atcs.offset_xy(x=offset, y=0)
    await latiss.take_flats(
        exptime=time, nflats=1, filter=filter_to_use, grating='empty_1', reason="Sky_Flat", program="Test-790")
    
    await atcs.offset_xy(x=0, y=offset)
    await latiss.take_flats(
        exptime=time, nflats=1, filter=filter_to_use, grating='empty_1', reason="Sky_Flat", program="Test-790")

## Image Assessment

The two next cells will serve to check the median, mean and std of the signal across an image. 
Edit the `day_obs` and `seq_num` in the `dataId` row. 

In [9]:
dataId = {'detector': 0, 'day_obs': 20230427, 'seq_num': 87}
exp = butler.get('raw', dataId=dataId)

In [10]:
foo = exp.getMaskedImage()
masked_array = np.ma.masked_array(foo.image.array, mask=foo.mask.array)
meta = exp.getMetadata()

logger.info(f"Observation ID: {meta['OBSID']}")
logger.info(f"    Median: {np.ma.median(masked_array)}")
logger.info(f"    Mean: {np.ma.mean(masked_array)}")
logger.info(f"    Std: {np.ma.std(masked_array)}")